In [ ]:
!pip install openai
!pip install watchdog

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 4.7 MB/s eta 0:00:00


In [ ]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-fv_g8ycf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-fv_g8ycf
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802823 sha256=bca010464c5ae64a854625b411a1594ce4023fac6e073926fc3b57ef163ec3ad
  Stored in directory: /tmp/pip-ephem-wheel-cache-wm3inj2u/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [ ]:
import whisper
import pandas as pd
import os
from watchdog.observers import Observer
from watchdog.events import FileSystemEventHandler
import time
import nltk
from nltk.tokenize import sent_tokenize
from openai import OpenAI
import numpy as np

In [ ]:


def transcreve_audio_pasta(audios_dir, transcricoes_existentes=None):
    # Carrega o modelo do Whisper
    modelo = whisper.load_model("large")

    # Verifica se existe algum DataFrame com transcrições anteriores
    if transcricoes_existentes is not None:
        df_existente = transcricoes_existentes
    else:
        df_existente = pd.DataFrame(columns=['Arquivo', 'Texto'])

    # Cria um conjunto com os arquivos já transcritos para evitar duplicações
    arquivos_transcritos = set(df_existente['Arquivo'].tolist())

    # Lista os arquivos de áudio na pasta
    arquivos_audio = [f for f in os.listdir(audios_dir) if f.endswith('.wav')]

    # Lista para armazenar novas transcrições
    novas_transcricoes = []

    # Loop para transcrever apenas os novos arquivos de áudio
    for arquivo in arquivos_audio:
        if arquivo not in arquivos_transcritos:
            caminho_arquivo = os.path.join(audios_dir, arquivo)
            print(f"Transcrevendo: {arquivo}")

            # Tenta realizar a transcrição e captura erros potenciais
            try:
                resposta = modelo.transcribe(caminho_arquivo)
                novas_transcricoes.append({'Arquivo': arquivo, 'Texto': resposta['text']})
            except Exception as e:
                print(f"Erro ao transcrever {arquivo}: {e}")

    # Adiciona as novas transcrições ao DataFrame existente
    df_novas_transcricoes = pd.DataFrame(novas_transcricoes)
    df_final = pd.concat([df_existente, df_novas_transcricoes], ignore_index=True)

    # Retorna o DataFrame final com todas as transcrições
    return df_final




In [ ]:
# Diretório de áudios correto: audios_dir

audios_dir = '/content/audios_dir'

# Exemplo de uso com DataFrame vazio ou existente
df_transcricoes_existentes = None

# Chama a função e obtém o DataFrame atualizado
df_transcricoes_atualizado = transcreve_audio_pasta(audios_dir, df_transcricoes_existentes)

# Mostra o DataFrame resultante
df_transcricoes_atualizado

/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device)


Transcrevendo: 2964909.wav
Transcrevendo: 2962505.wav
Transcrevendo: 2964097.wav
Transcrevendo: 2964578.wav
Transcrevendo: 2963930.wav
Transcrevendo: 2962184.wav
Transcrevendo: 2962219.wav
Transcrevendo: 2967409.wav
Transcrevendo: 2964496.wav
Transcrevendo: 2967344.wav
Transcrevendo: 2965051.wav
Transcrevendo: 2964672.wav
Transcrevendo: 2963977.wav
Transcrevendo: 2963210.wav
Transcrevendo: 2963371.wav
Transcrevendo: 2962307.wav
Transcrevendo: 2962972.wav
Transcrevendo: 2968585.wav
Transcrevendo: 2963510.wav
Transcrevendo: 2968175.wav
Transcrevendo: 2966180.wav
Transcrevendo: 2961972.wav
Transcrevendo: 2965979.wav
Transcrevendo: 2966683.wav
Transcrevendo: 2968253.wav
Transcrevendo: 2963847.wav
Transcrevendo: 2966138.wav
Transcrevendo: 2967671.wav
Transcrevendo: 2967631.wav
Transcrevendo: 2965622.wav
Transcrevendo: 2964228.wav
Transcrevendo: 2965935.wav
Transcrevendo: 2962946.wav
Transcrevendo: 2967602.wav
Transcrevendo: 2965900.wav
Transcrevendo: 2965853.wav
Transcrevendo: 2965821.wav
T

In [ ]:
df_transcricoes_atualizado_teste = pd.read_excel('df_transcricoes_atualizado_teste.xlsx')
df_transcricoes_atualizado_teste  = pd.DataFrame(df_transcricoes_atualizado_teste )
df_transcricoes_atualizado_teste

,Arquivo,Texto
0,2968253.wav,"Burbom Hotel Fortaleza, Satiko, boa tarde. Bo..."
1,2963847.wav,"Olá, você ligou para a Brico Bread. Estamos h..."
2,2965935.wav,"Alô? Alô, bom dia, Seu Leandro. Eu. É a Lília..."
3,2962946.wav,"Exótica, Fábio e Denise, bom dia. Bom dia, me..."
4,2965900.wav,Obrigado por telefonar para o Blue Tree Premi...
5,2965853.wav,"Alô. Alô, bom dia. Lília da empresa TOTS. Eu ..."
6,2965821.wav,"Bem-vindo a Suco Rico Citrus, para a unidade ..."
7,2967576.wav,Bem-vindo ao Bourbon Curitiba Hotel & Suites....
8,2964398.wav,"É que, Natália, boa tarde. Boa tarde, Natália..."
9,2966651.wav,"Alô? Alô, bom dia. Da empresa TOTOS falando. ..."


In [ ]:
from openai import OpenAI
client = OpenAI('''Colocar chave da API do chat gpt aqui''')

In [ ]:
def montar_pergunta(df_transcricoes_atualizado_teste, coluna_texto):
    pergunta_base = '''Preciso que você extraia as informações dos textos de acordo com a tabela de exemplo. Se atente em preencher cada coluna de forma correta, incluindo unidade, região, data de ligação e junte as respostas em apenas uma tabela:

Tabela de exemplo:
Nome    Nome da empresa Unidade da TOTVS    Região da TOTVS CNPJ    Sentimento  Data_Ligacao    Agilidade nos processos do suporte técnico: Atendimento dos agentes do suporte técnico: Probabilidade de recomendar a TOTVS Atendimento do Executivo de Vendas: Custos e valores dos produtos contratados:  Atendimento do financeiro:  Satisfação com a implantação do software:   Experiência com a atualização do software:  Atendimento realizado pela unidade TOTVS    Recomendação de empresa Comentários
Lucas Oliveira  TechSolutions Ltda  Unidade Pará    Norte   12345678901234  neutro  2024-01-08  7   6   7   7   7   6   6   7   7   InnovaTech Solutions    Atendimento adequado, mas esperava mais agilidade.
Juliana Costa   InfoTech S.A.   Unidade Ceará   Nordeste    98765432109876  positivo    2024-01-15  8   8   9   8   8   7   8   8   9   Alpha Systems Inc.  Recomendo pela eficiência e qualidade.
Rafael Santos   DataWorks Inc.  Unidade Brasília    Centro-Oeste    56789012345678  positivo    2024-02-02  9   9   10  8   9   8   9   9   9   FutureSoft Ltda Ótima experiência, sistema funcionando perfeitamente.
'''

    # Itera sobre o DataFrame e adiciona cada texto à pergunta
    for i, row in enumerate(df_transcricoes_atualizado_teste[coluna_texto], 1):
        pergunta_base += f'\nTexto {i} para análise: {row}'

    return pergunta_base

perguntar = montar_pergunta(df_transcricoes_atualizado_teste, 'Texto')


In [ ]:
print(perguntar)

Preciso que você extraia as informações dos textos de acordo com a tabela de exemplo. Se atente em preencher cada coluna de forma correta, incluindo unidade, região, data de ligação e junte as respostas em apenas uma tabela:

Tabela de exemplo:
Nome    Nome da empresa Unidade da TOTVS    Região da TOTVS CNPJ    Sentimento  Data_Ligacao    Agilidade nos processos do suporte técnico: Atendimento dos agentes do suporte técnico: Probabilidade de recomendar a TOTVS Atendimento do Executivo de Vendas: Custos e valores dos produtos contratados:  Atendimento do financeiro:  Satisfação com a implantação do software:   Experiência com a atualização do software:  Atendimento realizado pela unidade TOTVS    Recomendação de empresa Comentários
Lucas Oliveira  TechSolutions Ltda  Unidade Pará    Norte   12345678901234  neutro  2024-01-08  7   6   7   7   7   6   6   7   7   InnovaTech Solutions    Atendimento adequado, mas esperava mais agilidade.
Juliana Costa   InfoTech S.A.   Unidade Ceará   Nord

In [ ]:
completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": perguntar
        }
    ]
)

In [ ]:
resposta = completion.choices[0].message.content

In [ ]:
resposta

'Aqui está a tabela preenchida com as informações extraídas dos textos analisados:\n\n| Nome                | Nome da empresa                      | Unidade da TOTVS  | Região da TOTVS   | CNPJ                | Sentimento | Data_Ligacao | Agilidade nos processos do suporte técnico | Atendimento dos agentes do suporte técnico | Probabilidade de recomendar a TOTVS | Atendimento do Executivo de Vendas | Custos e valores dos produtos contratados | Atendimento do financeiro | Satisfação com a implantação do software | Experiência com a atualização do software | Atendimento realizado pela unidade TOTVS | Recomendação de empresa             | Comentários                                                                                                                                                     |\n|---------------------|-------------------------------------|-------------------|--------------------|---------------------|------------|--------------|-----------------------------------------

In [ ]:
from io import StringIO

# Utiliza StringIO para simular um arquivo e ler o conteúdo com pandas
data = StringIO(resposta)

# Lê  ira coluna
df = pd.read_csv(data, sep='|', engine='python', skipinitialspace=True, skiprows=1)

# Remove a primeira coluna (index) e limpa espaços nos nomes das colunas
df = df.iloc[1:, 1:19]  # Remove a primeira coluna
df.columns = [col.strip() for col in df.columns]  # Remove espaços em branco dos nomes das colunas

# Remove linhas que não têm dados reais (como linhas de observação)
df = df[df['Nome'].notna()]

df

,Nome,Nome da empresa,Unidade da TOTVS,Região da TOTVS,CNPJ,Sentimento,Data_Ligacao,Agilidade nos processos do suporte técnico,Atendimento dos agentes do suporte técnico,Probabilidade de recomendar a TOTVS,Atendimento do Executivo de Vendas,Custos e valores dos produtos contratados,Atendimento do financeiro,Satisfação com a implantação do software,Experiência com a atualização do software,Atendimento realizado pela unidade TOTVS,Recomendação de empresa,Comentários
1,Valtemberg Silva,Burbom Hotel Fortaleza,Unidade Fortaleza,Nordeste,1234567890162,positivo,2024-01-10,8,10,10,Não se aplica,8,9,Não participou,10,10,Não se aplica,Nenhum comentário ...
2,Walter Pereira,Brico Bread,Unidade São Paulo,Sudeste,0123456789000,neutro,2024-01-12,8,8,8,Não se aplica,8,Não se aplica,Não se aplica,9,Não se aplica,Não se aplica,Não houve participação direta. O sistema é com...
3,Leandro Rodrigues,Comab,Unidade Serra Gaúcha,Sul,0000123456789,neutro,2024-01-14,6,7,9,Não se aplica,5,Não se aplica,8,5,Não se aplica,Não se aplica,Lamentou a falta de controle em atualizações e...
4,Hélio Luíde Rodrigues,Casa Joca,Unidade Minas Gerais,Sudeste,0123456789012,neutro,2024-01-10,7,8,7,Não se aplica,Não se aplica,Não se aplica,9,Não se aplica,Não se aplica,Senti falta do módulo de eventos. ...,NaN
5,Roberto Martins,Sul Alimentação,Unidade Brasília,Centro-Oeste,0987654321098,positivo,2024-01-15,8,9,9,9,7,9,9,9,9,Não se aplica,"O suporte técnico é elogiado, com ênfase em su..."
6,Leandro Rodrigues da Silva,Brasilmed,Unidade Brasília,Centro-Oeste,0345789010000,neutro,2024-01-14,9,10,9,9,10,8,8,9,9,Não se aplica,Lamentou dificuldades de implantação e suporte...
7,Antônio Silva,Escola de Ensino Superior do Sertão,Unidade São Paulo,Sudeste,03475871000050,negativo,2024-01-10,5,5,5,Não se aplica,2,Não se aplica,10,10,Não se aplica,Não se aplica,O cliente reporta insatisfação com o suporte n...
8,Cláudia Oliveira,Baterias Crawl,Unidade São Paulo,Sudeste,2000000000000,neutro,2024-01-17,8,8,8,Não se aplica,8,Não se aplica,Não se aplica,6,7,Não se aplica,Relatou dificuldade nas atualizações do sistem...
9,Veronica Almeida,PA7 PEP Hotel,Unidade Rio de Janeiro,Sudeste,3000000000000,neutro,2024-01-20,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,Não se aplica,"Cliente não se sentiu confortável em avaliar, ...",NaN


In [ ]:
df_teste = df

In [ ]:
def process_table(table: pd.DataFrame) -> pd.DataFrame:
    df_teste = table.copy()  # Criar uma cópia do DataFrame original

    # Colunas que devem ser transformadas em notas
    note_columns = [
        "Agilidade nos processos do suporte técnico",
        "Atendimento dos agentes do suporte técnico",
        "Probabilidade de recomendar a TOTVS",
        "Atendimento do Executivo de Vendas",
        "Custos e valores dos produtos contratados",
        "Atendimento do financeiro",
        "Satisfação com a implantação do software",
        "Experiência com a atualização do software",
        "Atendimento realizado pela unidade TOTVS"
    ]

    # Função para converter para int ou NaN
    def convert_to_int(value):
        try:
            return int(value)
        except (ValueError, TypeError):
            return np.nan  # Retorna NaN se não for possível converter

    # Aplicar a conversão nas colunas de notas
    for col in note_columns:
        df_teste[col] = df_teste[col].apply(convert_to_int)

    return df_teste

df_teste_final = process_table(df_teste)


In [ ]:
df_teste_final = process_table(df)

In [ ]:
df_teste_final

,Nome,Nome da empresa,Unidade da TOTVS,Região da TOTVS,CNPJ,Sentimento,Data_Ligacao,Agilidade nos processos do suporte técnico,Atendimento dos agentes do suporte técnico,Probabilidade de recomendar a TOTVS,Atendimento do Executivo de Vendas,Custos e valores dos produtos contratados,Atendimento do financeiro,Satisfação com a implantação do software,Experiência com a atualização do software,Atendimento realizado pela unidade TOTVS,Recomendação de empresa,Comentários
1,Valtemberg Silva,Burbom Hotel Fortaleza,Unidade Fortaleza,Nordeste,1234567890162,positivo,2024-01-10,8.0,10.0,10.0,NaN,8.0,9.0,NaN,10.0,10.0,Não se aplica,Nenhum comentário ...
2,Walter Pereira,Brico Bread,Unidade São Paulo,Sudeste,0123456789000,neutro,2024-01-12,8.0,8.0,8.0,NaN,8.0,NaN,NaN,9.0,NaN,Não se aplica,Não houve participação direta. O sistema é com...
3,Leandro Rodrigues,Comab,Unidade Serra Gaúcha,Sul,0000123456789,neutro,2024-01-14,6.0,7.0,9.0,NaN,5.0,NaN,8.0,5.0,NaN,Não se aplica,Lamentou a falta de controle em atualizações e...
4,Hélio Luíde Rodrigues,Casa Joca,Unidade Minas Gerais,Sudeste,0123456789012,neutro,2024-01-10,7.0,8.0,7.0,NaN,NaN,NaN,9.0,NaN,NaN,Senti falta do módulo de eventos. ...,NaN
5,Roberto Martins,Sul Alimentação,Unidade Brasília,Centro-Oeste,0987654321098,positivo,2024-01-15,8.0,9.0,9.0,9.0,7.0,9.0,9.0,9.0,9.0,Não se aplica,"O suporte técnico é elogiado, com ênfase em su..."
6,Leandro Rodrigues da Silva,Brasilmed,Unidade Brasília,Centro-Oeste,0345789010000,neutro,2024-01-14,9.0,10.0,9.0,9.0,10.0,8.0,8.0,9.0,9.0,Não se aplica,Lamentou dificuldades de implantação e suporte...
7,Antônio Silva,Escola de Ensino Superior do Sertão,Unidade São Paulo,Sudeste,03475871000050,negativo,2024-01-10,5.0,5.0,5.0,NaN,2.0,NaN,10.0,10.0,NaN,Não se aplica,O cliente reporta insatisfação com o suporte n...
8,Cláudia Oliveira,Baterias Crawl,Unidade São Paulo,Sudeste,2000000000000,neutro,2024-01-17,8.0,8.0,8.0,NaN,8.0,NaN,NaN,6.0,7.0,Não se aplica,Relatou dificuldade nas atualizações do sistem...
9,Veronica Almeida,PA7 PEP Hotel,Unidade Rio de Janeiro,Sudeste,3000000000000,neutro,2024-01-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Cliente não se sentiu confortável em avaliar, ...",NaN


In [ ]:
df_teste_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9 entries, 1 to 9
Data columns (total 18 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Nome                                        9 non-null      object 
 1   Nome da empresa                             9 non-null      object 
 2   Unidade da TOTVS                            9 non-null      object 
 3   Região da TOTVS                             9 non-null      object 
 4   CNPJ                                        9 non-null      object 
 5   Sentimento                                  9 non-null      object 
 6   Data_Ligacao                                9 non-null      object 
 7   Agilidade nos processos do suporte técnico  8 non-null      float64
 8   Atendimento dos agentes do suporte técnico  8 non-null      float64
 9   Probabilidade de recomendar a TOTVS         8 non-null      float64
 10  Atendimento do Executiv

In [ ]:
df_teste_final.to_excel('df_teste_final.xlsx', index=False)